In [12]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [13]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [14]:
from spacy.lang.id import Indonesian
import spacy

nlp = Indonesian()  # use directly
nlp = spacy.blank('id')  # blank instance'

In [15]:
import re
import string
import time
from copy import deepcopy

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv('Analisis_sentimen_timnas_sepakbola_indonesia_di_era_STY.csv')

In [18]:
df.head()

,komentar,label
0,<username> Simulasi indonesia kembali dijajah ...,negatif
1,<username> <username> <username> keturunan lai...,negatif
2,<username> <username> uda botak keturunan penj...,negatif
3,"<username> <username> <username> Belum tentu, ...",negatif
4,<username> <username> <username> Lu nya aja yg...,negatif


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   komentar  398 non-null    object
 1   label     398 non-null    object
dtypes: object(2)
memory usage: 6.3+ KB


In [20]:
# Install dulu klo blm
#!pip install ekphrasis

In [21]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [22]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [23]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('rt'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

In [24]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

In [30]:
i = 0
final_string = []
s = ""
for text in df['komentar'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = lower(text)
    proc = change_stripe(text)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    proc = remove_rt(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)

    proc = proc.replace('username', '')  # Menghapus kata 'username'

    EachReviewText = proc
    final_string.append(EachReviewText)

In [31]:
df["step01"] = final_string

In [32]:
df.head(10)

,komentar,label,step01
0,<username> Simulasi indonesia kembali dijajah ...,negatif,simulasi indonesia kembali dijajah keturunan ...
1,<username> <username> <username> keturunan lai...,negatif,keturunan lain yang gak main bola emang bis...
2,<username> <username> uda botak keturunan penj...,negatif,uda botak keturunan penjajah sok si paling p...
3,"<username> <username> <username> Belum tentu, ...",negatif,belum tentu gak semuanya jadi pemain bola d...
4,<username> <username> <username> Lu nya aja yg...,negatif,lu nya aja yg bego makanya nonton sendiri t...
5,<username> Saya punya saran kepada <username> ...,negatif,saya punya saran kepada kita terkenal deng...
6,Ini STY ga coba asnawi jadi gelandang aja? VO2...,negatif,ini sty ga coba asnawi jadi gelandang aja vo m...
7,<username> <username> <username> Bukan masalah...,negatif,bukan masalah bnyak kesempatan tiap taun ju...
8,"<username> Biasa, sty klo tantrum kan gtu. Sem...",negatif,biasa sty klo tantrum kan gtu semua hal dibaw...
9,<username> Setelah pulang ke tanah air apakah ...,negatif,setelah pulang ke tanah air apakah akan dilem...


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   komentar  398 non-null    object
 1   label     398 non-null    object
 2   step01    398 non-null    object
dtypes: object(3)
memory usage: 9.5+ KB


In [34]:
df_hapus = df[~df['step01'].str.contains(" ")]

In [35]:
df_hapus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 243 to 394
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   komentar  2 non-null      object
 1   label     2 non-null      object
 2   step01    2 non-null      object
dtypes: object(3)
memory usage: 64.0+ bytes


In [36]:
df_hapus.head(10)

,komentar,label,step01
243,,positif,
394,\,positif,


In [37]:
df_new = df[~df.isin(df_hapus)].dropna()

In [38]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 0 to 397
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   komentar  396 non-null    object
 1   label     396 non-null    object
 2   step01    396 non-null    object
dtypes: object(3)
memory usage: 12.4+ KB


In [39]:
df_new

,komentar,label,step01
0,<username> Simulasi indonesia kembali dijajah ...,negatif,simulasi indonesia kembali dijajah keturunan ...
1,<username> <username> <username> keturunan lai...,negatif,keturunan lain yang gak main bola emang bis...
2,<username> <username> uda botak keturunan penj...,negatif,uda botak keturunan penjajah sok si paling p...
3,"<username> <username> <username> Belum tentu, ...",negatif,belum tentu gak semuanya jadi pemain bola d...
4,<username> <username> <username> Lu nya aja yg...,negatif,lu nya aja yg bego makanya nonton sendiri t...
...,...,...,...
392,<username> <username> 3thun llu gk di naturali...,positif,thun llu gk di naturalisasi mgkin msih trau...
393,<username> <username> 3 tahun lalu indonesia m...,positif,tahun lalu indonesia memang ga gencar buat...
395,Naturalisasi pemain keturunan menjadi langkah ...,positif,naturalisasi pemain keturunan menjadi langkah ...
396,semua persyaratan yang diperlukan untuk proses...,positif,semua persyaratan yang diperlukan untuk proses...


In [40]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [41]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [42]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [43]:
df_new.head(10)

,komentar,label,step01,tokens
0,<username> Simulasi indonesia kembali dijajah ...,negatif,simulasi indonesia kembali dijajah keturunan ...,"[simulasi, indonesia, kembali, dijajah, keturu..."
1,<username> <username> <username> keturunan lai...,negatif,keturunan lain yang gak main bola emang bis...,"[keturunan, lain, yang, gak, main, bola, emang..."
2,<username> <username> uda botak keturunan penj...,negatif,uda botak keturunan penjajah sok si paling p...,"[uda, botak, keturunan, penjajah, sok, si, pal..."
3,"<username> <username> <username> Belum tentu, ...",negatif,belum tentu gak semuanya jadi pemain bola d...,"[belum, tentu, gak, semuanya, jadi, pemain, bo..."
4,<username> <username> <username> Lu nya aja yg...,negatif,lu nya aja yg bego makanya nonton sendiri t...,"[lu, nya, aja, yg, bego, makanya, nonton, send..."
5,<username> Saya punya saran kepada <username> ...,negatif,saya punya saran kepada kita terkenal deng...,"[saya, punya, saran, kepada, kita, terkenal, d..."
6,Ini STY ga coba asnawi jadi gelandang aja? VO2...,negatif,ini sty ga coba asnawi jadi gelandang aja vo m...,"[ini, sty, ga, coba, asnawi, jadi, gelandang, ..."
7,<username> <username> <username> Bukan masalah...,negatif,bukan masalah bnyak kesempatan tiap taun ju...,"[bukan, masalah, bnyak, kesempatan, tiap, taun..."
8,"<username> Biasa, sty klo tantrum kan gtu. Sem...",negatif,biasa sty klo tantrum kan gtu semua hal dibaw...,"[biasa, sty, klo, tantrum, kan, gtu, semua, ha..."
9,<username> Setelah pulang ke tanah air apakah ...,negatif,setelah pulang ke tanah air apakah akan dilem...,"[setelah, pulang, ke, tanah, air, apakah, akan..."


In [44]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [45]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

C:\Users\ASUS TUFF\AppData\Local\Temp\ipykernel_4504\2408214049.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
C:\Users\ASUS TUFF\AppData\Local\Temp\ipykernel_4504\2408214049.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [46]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [47]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [48]:
df_new["step02"] = final_string_tokens

In [49]:
df_new.head(10)

,komentar,label,step01,tokens,final_tokens,step02
0,<username> Simulasi indonesia kembali dijajah ...,negatif,simulasi indonesia kembali dijajah keturunan ...,"[simulasi, indonesia, kembali, dijajah, keturu...","[simulasi, indonesia, kembali, dijajah, keturu...",simulasi indonesia kembali dijajah keturunan b...
1,<username> <username> <username> keturunan lai...,negatif,keturunan lain yang gak main bola emang bis...,"[keturunan, lain, yang, gak, main, bola, emang...","[keturunan, lain, yang, gak, main, bola, emang...",keturunan lain yang gak main bola emang bisa d...
2,<username> <username> uda botak keturunan penj...,negatif,uda botak keturunan penjajah sok si paling p...,"[uda, botak, keturunan, penjajah, sok, si, pal...","[uda, botak, keturunan, penjajah, sok, si, pal...",uda botak keturunan penjajah sok si paling pah...
3,"<username> <username> <username> Belum tentu, ...",negatif,belum tentu gak semuanya jadi pemain bola d...,"[belum, tentu, gak, semuanya, jadi, pemain, bo...","[belum, tentu, gak, semuanya, jadi, pemain, bo...",belum tentu gak semuanya jadi pemain bola dan ...
4,<username> <username> <username> Lu nya aja yg...,negatif,lu nya aja yg bego makanya nonton sendiri t...,"[lu, nya, aja, yg, bego, makanya, nonton, send...","[lu, nya, aja, yg, bego, makanya, nonton, send...",lu nya aja yg bego makanya nonton sendiri tolo...
5,<username> Saya punya saran kepada <username> ...,negatif,saya punya saran kepada kita terkenal deng...,"[saya, punya, saran, kepada, kita, terkenal, d...","[saya, punya, saran, kepada, kita, terkenal, d...",saya punya saran kepada kita terkenal dengan p...
6,Ini STY ga coba asnawi jadi gelandang aja? VO2...,negatif,ini sty ga coba asnawi jadi gelandang aja vo m...,"[ini, sty, ga, coba, asnawi, jadi, gelandang, ...","[ini, sty, ga, coba, asnawi, jadi, gelandang, ...",ini sty ga coba asnawi jadi gelandang aja vo m...
7,<username> <username> <username> Bukan masalah...,negatif,bukan masalah bnyak kesempatan tiap taun ju...,"[bukan, masalah, bnyak, kesempatan, tiap, taun...","[bukan, masalah, bnyak, kesempatan, tiap, taun...",bukan masalah bnyak kesempatan tiap taun juga ...
8,"<username> Biasa, sty klo tantrum kan gtu. Sem...",negatif,biasa sty klo tantrum kan gtu semua hal dibaw...,"[biasa, sty, klo, tantrum, kan, gtu, semua, ha...","[biasa, sty, klo, tantrum, kan, gtu, semua, ha...",biasa sty klo tantrum kan gtu semua hal dibawa...
9,<username> Setelah pulang ke tanah air apakah ...,negatif,setelah pulang ke tanah air apakah akan dilem...,"[setelah, pulang, ke, tanah, air, apakah, akan...","[setelah, pulang, ke, tanah, air, apakah, akan...",setelah pulang ke tanah air apakah akan dilemp...


In [50]:
df.to_csv('clean_dataset2.csv',sep=";")